# Jupiter notebook for the eighth seminar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import trange, tqdm
%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams
rcParams['figure.figsize'] = 8,5
%matplotlib inline

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
print("Version of tensorflow:",tf.__version__)
import sys
from IPython.display import clear_output

Version of tensorflow: 2.7.0


In [ ]:
# Downloading dataset
data_train = tfds.load(name="fashion_mnist", split="train").prefetch(60000).cache()
data_test  = tfds.load(name="fashion_mnist", split="test" ).prefetch(10000).cache()

## Exercise 1

Let's make a model with preprocessing part out of it:

In [ ]:
num_hidden = 64
modelWithoutPreprocessing = tf.keras.Sequential([                                             
  tf.keras.layers.Dense(num_hidden, activation='relu', name='hidden_layer'),
  tf.keras.layers.Dense(10, name='output_layer')
])

In [ ]:
modelWithoutPreprocessing.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])

In [ ]:
def unwrap(x):
  # Unwraps dataset to the appropriate form
  return (x['image'], x['label'])

def preprocessing(x):
  # Function to put the preprocessing part out of the model
  img = tf.reshape(tf.cast(x['image'], 'float32')/255., [-1, 28**2])
  lbl = x['label']
  return (img, lbl)

In [ ]:
batch_size = 1024
num_epochs = 10
modelWithoutPreprocessing.fit(x=data_train.shuffle(60000).batch(batch_size).map(preprocessing),
          epochs=num_epochs,
          validation_data=data_test.batch(4096).map(preprocessing))

Epoch 1/10
59/59 [==============================] - 1s 13ms/step - loss: 0.2934 - sparse_categorical_accuracy: 0.8959 - val_loss: 0.3648 - val_sparse_categorical_accuracy: 0.8728
Epoch 2/10
59/59 [==============================] - 1s 13ms/step - loss: 0.2906 - sparse_categorical_accuracy: 0.8962 - val_loss: 0.3679 - val_sparse_categorical_accuracy: 0.8730
Epoch 3/10
59/59 [==============================] - 1s 13ms/step - loss: 0.2888 - sparse_categorical_accuracy: 0.8977 - val_loss: 0.3567 - val_sparse_categorical_accuracy: 0.8755
Epoch 4/10
59/59 [==============================] - 1s 13ms/step - loss: 0.2842 - sparse_categorical_accuracy: 0.8991 - val_loss: 0.3569 - val_sparse_categorical_accuracy: 0.8749
Epoch 5/10
59/59 [==============================] - 1s 12ms/step - loss: 0.2828 - sparse_categorical_accuracy: 0.8992 - val_loss: 0.3592 - val_sparse_categorical_accuracy: 0.8762
Epoch 6/10
59/59 [==============================] - 1s 13ms/step - loss: 0.2795 - sparse_categorical_accu

In [ ]:
modelWithoutPreprocessing.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer (Dense)        (None, 1, 64)             50240     
                                                                 
 output_layer (Dense)        (None, 1, 10)             650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


## Exercise 2

Let's train a model with more layers:

In [ ]:
num_first_hidden = 64
num_second_hidden = 128
num_third_hidden = 64
modelWithMoreLayers = tf.keras.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.reshape(x / 255., (-1, 28*28)),
      input_shape=(28, 28, 1), name='preprocessing'),                                   
  tf.keras.layers.Dense(num_first_hidden, activation='relu', name='1st_hidden_layer'),
  tf.keras.layers.Dense(num_second_hidden, activation='relu', name='2nd_hidden_layer'),
  tf.keras.layers.Dense(num_third_hidden, activation='relu', name='3rd_hidden_layer'),
  tf.keras.layers.Dense(10, name='output_layer')
])

In [ ]:
modelWithMoreLayers.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])

In [ ]:
batch_size = 1024
num_epochs = 10
modelWithMoreLayers.fit(x=data_train.map(unwrap).shuffle(60000).batch(batch_size),
          epochs=num_epochs,
          validation_data=data_test.map(unwrap).batch(4096))

Epoch 1/10
59/59 [==============================] - 2s 19ms/step - loss: 1.0110 - sparse_categorical_accuracy: 0.6520 - val_loss: 0.6040 - val_sparse_categorical_accuracy: 0.7761
Epoch 2/10
59/59 [==============================] - 1s 17ms/step - loss: 0.5147 - sparse_categorical_accuracy: 0.8205 - val_loss: 0.4971 - val_sparse_categorical_accuracy: 0.8234
Epoch 3/10
59/59 [==============================] - 1s 16ms/step - loss: 0.4398 - sparse_categorical_accuracy: 0.8457 - val_loss: 0.4535 - val_sparse_categorical_accuracy: 0.8375
Epoch 4/10
59/59 [==============================] - 1s 17ms/step - loss: 0.4082 - sparse_categorical_accuracy: 0.8555 - val_loss: 0.4292 - val_sparse_categorical_accuracy: 0.8458
Epoch 5/10
59/59 [==============================] - 1s 17ms/step - loss: 0.3857 - sparse_categorical_accuracy: 0.8634 - val_loss: 0.4153 - val_sparse_categorical_accuracy: 0.8520
Epoch 6/10
59/59 [==============================] - 1s 16ms/step - loss: 0.3677 - sparse_categorical_accu

In [ ]:
modelWithMoreLayers.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocessing (Lambda)      (None, 784)               0         
                                                                 
 1st_hidden_layer (Dense)    (None, 64)                50240     
                                                                 
 2nd_hidden_layer (Dense)    (None, 128)               8320      
                                                                 
 3rd_hidden_layer (Dense)    (None, 64)                8256      
                                                                 
 output_layer (Dense)        (None, 10)                650       
                                                                 
Total params: 67,466
Trainable params: 67,466
Non-trainable params: 0
_________________________________________________________________
